In [14]:
from asitop.utils import parse_gpu_metrics, parse_powermetrics, run_powermetrics_process, parse_cpu_metrics

In [15]:
cpu_metrics_dict, gpu_metrics_dict, thermal_pressure, bandwidth_metrics, timestamp = parse_powermetrics('powermetrics.txt', timecode='')
gpu_metrics_dict

{'freq_MHz': 388, 'active': 3}

In [16]:
import plistlib
import pandas as pd

with open('powermetrics.txt', 'rb') as fp:
    data = fp.read()

metrics_list = []    
for d in data.split(b'\x00'):
    d = plistlib.loads(d)
    
    cpu_m = parse_cpu_metrics(d)

    metrics_list.append({
        "GPU_freq_MHz": int(d['gpu']["freq_hz"]),
        "GPU_active": int((1 - d['gpu']["idle_ratio"])*100),
        **cpu_m
        
    })

df = pd.DataFrame(metrics_list)
df.head()

,GPU_freq_MHz,GPU_active,E-Cluster_freq_Mhz,E-Cluster_active,E-Cluster0_freq_Mhz,E-Cluster0_active,E-Cluster1_freq_Mhz,E-Cluster1_active,P0-Cluster_freq_Mhz,P0-Cluster_active,...,P-Cluster9_freq_Mhz,P-Cluster9_active,e_core,p_core,P-Cluster_active,P-Cluster_freq_Mhz,ane_W,cpu_W,gpu_W,package_W
0,432,6,1344,49,1424,38,1425,37,1192,29,...,1902,0,"[0, 1]","[2, 3, 4, 5, 6, 7, 8, 9]",23,1192,0.0,0.595,0.039,0.628634
1,752,39,1545,56,1645,44,1649,44,1971,59,...,2897,3,"[0, 1]","[2, 3, 4, 5, 6, 7, 8, 9]",42,1971,0.0,3.247,1.870,4.953540
2,389,95,1530,54,1543,43,1553,42,3206,77,...,2919,0,"[0, 1]","[2, 3, 4, 5, 6, 7, 8, 9]",58,3206,0.0,4.671,1.509,6.039180
3,388,95,1404,63,1459,51,1450,51,3206,71,...,2624,0,"[0, 1]","[2, 3, 4, 5, 6, 7, 8, 9]",60,3206,0.0,4.669,1.519,6.049740
4,388,96,1412,63,1455,50,1483,48,3207,69,...,3228,0,"[0, 1]","[2, 3, 4, 5, 6, 7, 8, 9]",59,3207,0.0,4.658,1.562,6.082630


In [17]:
df.columns

Index(['GPU_freq_MHz', 'GPU_active', 'E-Cluster_freq_Mhz', 'E-Cluster_active',
       'E-Cluster0_freq_Mhz', 'E-Cluster0_active', 'E-Cluster1_freq_Mhz',
       'E-Cluster1_active', 'P0-Cluster_freq_Mhz', 'P0-Cluster_active',
       'P-Cluster2_freq_Mhz', 'P-Cluster2_active', 'P-Cluster3_freq_Mhz',
       'P-Cluster3_active', 'P-Cluster4_freq_Mhz', 'P-Cluster4_active',
       'P-Cluster5_freq_Mhz', 'P-Cluster5_active', 'P1-Cluster_freq_Mhz',
       'P1-Cluster_active', 'P-Cluster6_freq_Mhz', 'P-Cluster6_active',
       'P-Cluster7_freq_Mhz', 'P-Cluster7_active', 'P-Cluster8_freq_Mhz',
       'P-Cluster8_active', 'P-Cluster9_freq_Mhz', 'P-Cluster9_active',
       'e_core', 'p_core', 'P-Cluster_active', 'P-Cluster_freq_Mhz', 'ane_W',
       'cpu_W', 'gpu_W', 'package_W'],
      dtype='object')

In [18]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots


orig_df = df.copy()
df = orig_df[:90]

# Create a figure with two subplots
fig = make_subplots(rows=2, cols=1, shared_xaxes=True)

# Add traces
fig.add_trace(go.Scatter(x=df.index, y=df['P-Cluster_active'], mode='lines', name='CPU P-Cluster %'), row=1, col=1)
fig.add_trace(go.Scatter(x=df.index, y=df['E-Cluster_active'], mode='lines', name='CPU E-Cluster %'), row=1, col=1)
fig.add_trace(go.Scatter(x=df.index, y=df.GPU_active, mode='lines', name='Active GPU %'), row=1, col=1)
fig.add_trace(go.Scatter(x=df.index, y=df.GPU_freq_MHz, mode='lines', name='GPU freq (MHz)'), row=2, col=1)

# Update layout if needed (titles, axis labels etc.)
# fig.update_layout(height=400, width=800, title_text="CPU/GPU usage for CPU-based torch model inference")
fig.update_layout(height=400, width=800, title_text="CPU/GPU usage for MLX model inference")
fig.update_xaxes(title_text="Time (seconds)", row=2, col=1)  # Update only the bottom x-axis

fig.update_yaxes(range=[0, 100], row=1, col=1)

# Show the figure
fig.show()

